# Laboratorium 5
## Jakub Sacha

In [ ]:
surname = "Sacha"
print(f"Numer datasetu do pobrania i uporządkowania: {len(surname) % 5 + 1}")

Numer datasetu do pobrania i uporządkowania: 1


### Poniższy skrypt wycina z pliku weather.txt dane z roku 2010 i zapisuje je do pliku cut_weather.txt w folderze analysis_data.

In [ ]:
import os 
import re

# not the best way to grab that data but it will do for now
go_up_by = len('command_files')
original_data_filepath = os.getcwd()[:-go_up_by] + 'original_data/weather.txt'
cut_data_filepath = os.getcwd()[:-go_up_by] + 'analysis_data/cut_weather.txt'
tidy_data_filepath = os.getcwd()[:-go_up_by] + 'analysis_data/tidied_weather.csv'
data_appendix_filepath = os.getcwd()[:-go_up_by] + 'documents/data_appendix.csv'

# cut the data to the part we are interested in (5 months in 2010)
interesting_data_id_pattern = r'MX0000170042010[01][0-9][A-Z]{4}'
cut_data = ""
with open(original_data_filepath, 'r') as f:
    for line in f.readlines():
        if re.findall(interesting_data_id_pattern, line):
            cut_data += line

# save the narrowed data  
with open(cut_data_filepath, 'w') as f:
    f.write(cut_data)


### Ten fragment notebooka odpowiedzialny jest za wczytanie interesujących nas danych oraz wstępne ich pogrupowanie. Wynikiem tych operacji jest poglądowy dataframe.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

days_range = range(1, 32)

column_names = ["id", "year", "month", "variable name"] + [f"d{i}" for i in days_range]

weather_station_id = "MX000017004"
rows = []

with open(cut_data_filepath, "r") as dataset:
    for line in dataset.readlines():
        start = len(weather_station_id)

        year = line[start:start+4]
        month = line[start+len(year):start+len(year)+2]
        var_name = line[start+len(year)+len(month):start+len(year)+len(month)+4]

        days_data_idx = len(weather_station_id) + len(year) + len(month) + len(var_name)

        days_data = line[days_data_idx:]
        # I search for every number in the part that's representing daily measurements and check if it's a valid value(I assume '9999' is a NaN value)
        days_cleaned_data = [float(number) if number != '9999' else np.NaN for number in re.findall('[0-9]+', days_data) ]

        rows.append([weather_station_id, year, month, var_name] + days_cleaned_data)

df_before_tiding = pd.DataFrame(rows, columns=column_names)
df_before_tiding.head(10)

,id,year,month,variable name,d1,d2,d3,d4,d5,d6,...,d22,d23,d24,d25,d26,d27,d28,d29,d30,d31
0,MX000017004,2010,01,TMAX,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,278.0,NaN
1,MX000017004,2010,01,TMIN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,145.0,NaN
2,MX000017004,2010,01,PRCP,NaN,NaN,NaN,NaN,0.0,NaN,...,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN
3,MX000017004,2010,02,TMAX,NaN,273.0,241.0,NaN,NaN,NaN,...,NaN,299.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,MX000017004,2010,02,TMIN,NaN,144.0,144.0,NaN,NaN,NaN,...,NaN,107.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,MX000017004,2010,02,PRCP,NaN,0.0,0.0,NaN,NaN,0.0,...,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
6,MX000017004,2010,03,TMAX,NaN,NaN,NaN,NaN,321.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,MX000017004,2010,03,TMIN,NaN,NaN,NaN,NaN,142.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,MX000017004,2010,03,PRCP,NaN,0.0,NaN,0.0,0.0,0.0,...,NaN,0.0,NaN,NaN,0.0,0.0,NaN,NaN,0.0,NaN
9,MX000017004,2010,04,TMAX,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,363.0,NaN,NaN,NaN,NaN


### Następnie segreguję dane według zasad tidy data. Dane zapisane zostają do tidied_weather.csv w folderze analysis_data oraz zostaje wyświetlony poglądowy dataframe.

In [ ]:
# after reading and organising data from the file I can apply tidy data rules
column_names = ["weather_station_id", "date", "TMAX", "TMIN", "PRCP"]
tidied_rows = []
# I take advantage of the data order in weather.txt
for (tmax_row, tmin_row, prcp_row) in zip(rows[0::3], rows[1::3], rows[2::3]):
    ID = tmax_row[0]
    for i, day in enumerate(days_range):
        if int(day) < 10:
            day = "0" + str(day)
        year, month = tmax_row[1], tmax_row[2]
        date = f'{year}-{month}-{day}'

        tmax_value = float(tmax_row[i+4]) / 10 if tmax_row[i+4] is not np.NaN else np.NaN
        tmin_value = float(tmin_row[i+4]) / 10 if tmin_row[i+4] is not np.NaN else np.NaN
        prcp_value = float(prcp_row[i+4]) / 10 if prcp_row[i+4] is not np.NaN else np.NaN

        if tmax_value > tmin_value:
            # in case when tmax is greater than tmin an error must have occurred so I don't record those values
            tidied_rows.append([ID, date, tmax_value, tmin_value, prcp_value])

tidied_df = pd.DataFrame(tidied_rows, columns=column_names)
tidied_df.dropna(inplace=True)
tidied_df.reset_index(drop=True, inplace=True)

# save data to csv
with open(tidy_data_filepath, 'w') as output:
    output.write(tidied_df.to_csv(index=False))
# make a copy for data appendix
with open(data_appendix_filepath, 'w') as data_appendix_file:
    data_appendix_file.write(tidied_df.to_csv(index=False))
# show how the data looks now
tidied_df.head(10)

,weather_station_id,date,TMAX,TMIN,PRCP
0,MX000017004,2010-01-30,27.8,14.5,0.0
1,MX000017004,2010-02-02,27.3,14.4,0.0
2,MX000017004,2010-02-03,24.1,14.4,0.0
3,MX000017004,2010-02-11,29.7,13.4,0.0
4,MX000017004,2010-02-23,29.9,10.7,0.0
5,MX000017004,2010-03-05,32.1,14.2,0.0
6,MX000017004,2010-03-10,34.5,16.8,0.0
7,MX000017004,2010-03-16,31.1,17.6,0.0
8,MX000017004,2010-04-27,36.3,16.7,0.0
9,MX000017004,2010-05-27,33.2,18.2,0.0


### Przy każdej z tych wartości zarejestrowana została flaga 'S', która oznacza 'failed spatial consistency check' zgodnie z dokumentacją danych z Global Historical Climatology Network, zatem użyteczność danych zgromadzonych jest w pewien sposób ograniczona.